In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [110]:
# main_folder = r'D:\Test\20220427_M25_PSF_Alignment' #Middle Wiggle

#Scope Parameters
# FOV = 50e-6
cam_px = 5.86e-6
totalmag = 37.5
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

1.5626666666666666e-07
0.341296928327645
12.79863481228669


In [111]:
#Load Only center Camera to see if good FOV and signal
try:
    center_planes = calibration.load_dataset(main_folder,num_cams=4,px_depth='uint8')
    print("8bit")
except: 
    print("16bit")
    center_planes = calibration.load_dataset(main_folder,num_cams=4,px_depth='uint16')


8bit


In [112]:
# viewer = napari.Viewer()
viewer.add_image(center_planes, name='center_planes',scale=[z_scale,1,1])

<Image layer 'center_planes [1]' at 0x25758455a00>

In [32]:
try:
    stack= calibration.load_dataset(main_folder,'uint8')
    print("8bit")
except: 
    print("16bit")
    stack = calibration.load_dataset(main_folder,'uint16')

16bit


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [5]:
# Load offsets from desired folder 
main_folder_offset = r'E:\Ed\PSF_defocus\20220404_M25_tamronArray_mfgideal\20220405_M25_PSF2'
offsets_file_load = os.path.join(main_folder_offset,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

### Align Datasets with Known Offsets


In [6]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(500, 25, 600, 960)


In [7]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*0.86174

print(da_stack_align)

dask.array<setitem, shape=(500, 25, 600, 960), dtype=uint8, chunksize=(125, 25, 150, 160), chunktype=numpy.ndarray>


In [8]:
viewer= napari.Viewer()

In [9]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1])

C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\napari\utils\events\event.py:283: RuntimeWarning: coroutine 'get_msg' was never awaited
  self._callbacks: List[Union[Callback, CallbackRef]] = []
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\napari\utils\events\event.py:768: RuntimeWarning: coroutine 'get_msg' was never awaited
  _kwargs = self.default_args.copy()


<Image layer 'dask_align' at 0x1b22a0fc7f0>

In [18]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
# ij.ui().show('PSF', ij_stack2)

### CROP PSF TO ANALYZE


In [10]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[ 13.           0.         171.66279066 245.7774276 ]
 [ 13.           0.         171.66279066 533.73394949]
 [ 13.           0.         459.61931255 533.73394949]
 [ 13.           0.         459.61931255 245.7774276 ]]
[[172 246]
 [460 534]]
(2, 2)


In [11]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

13 0


<Image layer 'crop' at 0x1a80df2be50>

### Image J Part

In [12]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init('net.imagej:imageJ:2.1.0',mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [13]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_img))
ij.ui().show('PSF', ij_stack2)

In [17]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(250, 288, 288, 25)


In [39]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1])

(250, 25, 288, 288)


<Image layer 'crop [1]' at 0x1ac68ab14f0>

#### Save TIFF FIles

In [45]:
import tifffile

filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
print(filepath_save)
tifffile.imsave(filepath_save,anp2_da)

E:\Ed\Data\20220405_drosophila_egg_larvae\20220405_M25_larvae3b\crop_section.ome.tif


C:\Users\Callisto\AppData\Local\Temp\ipykernel_8980\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [4]:
viewer = napari.Viewer()

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [7]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)
viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**8-1))

(250, 25, 288, 288)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x19786245cd0>

C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

c


In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'